# Final Project: Segmentation of satellite images

## 1 Data

In [3]:
from osgeo import gdal
import numpy as np
import random as rd
import os

# get images:
data = gdal.Open(os.path.join("data", "satellite_image.tif"))
segmentation = gdal.Open(os.path.join("data", "final.tif"))

# get coordinate info:
ulx, xres, xskew, uly, yskew, yres = data.GetGeoTransform()

# define grid for splits in pixels:
grid_x_px = np.arange(0, data.RasterXSize, step=1000)
grid_y_px = np.arange(0, data.RasterYSize, step=1000)

# calculate grid for splits in coordinates:
grid_x_co = ulx + np.arange(0, data.RasterXSize, step=1000) * xres
grid_y_co = uly + np.arange(0, data.RasterYSize, step=1000) * yres

# set training set, validation set and test ratio:
data_set_folders = ["training", "validation", "test"]
data_set_ratios = [0.7, 0.15, 0.15]

# create directories if needed:
for folder in data_set_folders:
    if not os.path.isdir(os.path.join("data", folder)):
        os.mkdir(os.path.join("data", folder))
        os.mkdir(os.path.join("data", folder, "images"))
        os.mkdir(os.path.join("data", folder, "annotations"))

# split images:
for ix in range(len(grid_x_co) - 1):
    for iy in range(len(grid_y_co) - 1):
        # choose data set randomly:
        data_set_folder = rd.choices(data_set_folders, weights=data_set_ratios)[0]
        # define setting for translation:
        translate_options = gdal.TranslateOptions(gdal.ParseCommandLine(f"-outsize 1000 1000 -projwin {grid_x_co[ix]} {grid_y_co[iy]} {grid_x_co[ix + 1]} {grid_y_co[iy + 1]}"))
        # split satellite image:
        gdal.Translate(os.path.join("data", data_set_folder, "images", f"img_{ix}_{iy}.tif"), data, options = translate_options)
        # split annotations:
        gdal.Translate(os.path.join("data", data_set_folder, "annotations", f"img_{ix}_{iy}.tif"), segmentation, options = translate_options)

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, Cropping2D
from tensorflow.keras import Model

In [4]:
class ConvBlock(Model):

    def __init__(self, filters, kernel_size, pool_size):
        super(ConvBlock, self).__init__()
        self.conv1 = Conv2D(filters, kernel_size, activation="relu")
        self.conv2 = Conv2D(filters, kernel_size, activation="relu")
        self.max_pool = MaxPool2D(pool_size)

    def call(self, x):
        x = self.conv1(x)
        y = self.conv2(x)
        x = self.max_pool(y)
        return x, y

class UpConvBlock(Model):

    def __init__(self, filters, conv_kernel_size, up_conv_kernel_size, crop_size):
        super(UpConvBlock, self).__init__()
        self.upconv = Conv2DTranspose(filters, up_conv_kernel_size, activation="relu")
        self.crop = Cropping2D((crop_size, crop_size))
        self.conv1 = Conv2D(filters, conv_kernel_size, activation="relu")
        self.conv2 = Conv2D(filters, conv_kernel_size, activation="relu")

    def call(self, x, y):
        x = self.upconv(x)
        y = self.crop(y)
        x = tf.concat([y, x])
        x = self.conv1(x)
        x = self.conv2(x)
        return x

In [8]:

class UNet(Model):

    def __init__(self, input_dim = (572, 572), depth = 4, filters = 64, conv_kernel_size = 3, pool_size = 2, up_conv_kernel_size = 2, n_classes = 1):
        super(UNet, self).__init__()
        self.conv_blocks = []
        output_dim = list(input_dim)
        for i in range(depth):
            self.conv_blocks.append(ConvBlock(2**i * filters, conv_kernel_size, pool_size))
            output_dim = ((output_dim - 2 * conv_kernel_size + 2) - pool_size) / pool_size + 1
            if round(output_dim[0]) != output_dim[0] or round(output_dim[1]) != output_dim[1]:
                print("Warning: Output dimensions are rounded! Please check your inputs if this was not intended.")
            output_dim = [int(output_dim[0]), int(output_dim[1])]
        self.conv1 = Conv2D((depth+1) * filters, conv_kernel_size, activation="relu")
        self.conv2 = Conv2D((depth+1) * filters, conv_kernel_size, activation="relu")
        output_dim = output_dim - 2 * conv_kernel_size + 2
        self.up_conv_blocks = []
        for i in range(depth):
            input_dim = output_dim - 1 + up_conv_kernel_size
            self.up_conv_blocks.append(UpConvBlock(2**(depth-i) * filters, conv_kernel_size, up_conv_kernel_size, crop_size))
        self.conv3 = Conv2D(1, n_classes, activation="relu")

    def call(self, x):
        y = []
        for conv in self.conv_blocks:
            x, y = conv(x)
        x = self.conv1(x)
        x = self.conv2(x)
        for i, conv in enumerate(self.up_conv_blocks):
            x = conv(x, y[i])
        x = self.conv3(x)
        return x

In [9]:
model = UNet()